In [1]:
import pandas as pd
import numpy as np

In [2]:
db = pd.read_csv('spam-sms.csv', encoding = 'latin-1')
db.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)
db['bait'] = db['v1'].map({'ham': 0, 'spam': 1})
db['title'] = db['v2']
db.drop(['v1','v2'],axis=1,inplace=True)
print(db.head())
print(db.tail())

   bait                                              title
0     0  Go until jurong point, crazy.. Available only ...
1     0                      Ok lar... Joking wif u oni...
2     1  Free entry in 2 a wkly comp to win FA Cup fina...
3     0  U dun say so early hor... U c already then say...
4     0  Nah I don't think he goes to usf, he lives aro...
      bait                                              title
5567     1  This is the 2nd time we have tried 2 contact u...
5568     0              Will Ì_ b going to esplanade fr home?
5569     0  Pity, * was in mood for that. So...any other s...
5570     0  The guy did some bitching but I acted like i'd...
5571     0                         Rofl. Its true to its name


In [4]:
from sklearn.model_selection import train_test_split
X = db.title
y = db.bait
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

pipeline1 = Pipeline([('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('mnb', MultinomialNB())])

In [7]:
pipeline1.fit(X, y)
predicted = pipeline1.predict(X_test)
print (np.mean(predicted == y_test))

0.9777458722182341


In [12]:
pipeline1.predict(["Free entry in 2 a wkly comp to win FA Cup final"])[0]

1

In [14]:
pipeline1.predict_proba(["Free entry in 2 a wkly comp to win FA Cup final"])[0]

array([0.09000495, 0.90999505])

In [15]:
from sklearn.externals import joblib
import pickle

In [16]:
joblib.dump(pipeline1, 'spammailmodelsklearn.pkl')

['spammailmodelsklearn.pkl']

In [18]:
m1=joblib.load('spammailmodelsklearn.pkl')

In [19]:
pred_score = m1.predict_proba(['You won’t believe how these 9 shocking clickbaits work!'])[0]
pred_score

array([0.96492392, 0.03507608])

In [20]:
prob = {'Clickbate': pred_score[1], 'Not-Clickbate': pred_score[0]}

In [21]:
def score(input_string):
   
    pred_score = m1.predict_proba([input_string])[0]
    if ((pred_score[1]*100 > 40) & ((pred_score[1]*100 < 60))):
        flag = 'Maybe Baity'
        color = 'warn-color'
    elif (pred_score[1]*100 > 60):
        flag = 'Looks Safe'
        color = 'safe-color'
    else:
        flag = 'Sounds Baity'
        color = 'danger-color'

    prob = {'clickbate-prob': pred_score[1], 'notClickbate-prob': pred_score[0], 'flag': flag, 'color':color}
    return prob

In [24]:
a = score("Free entry in 2 a wkly comp to win FA Cup final")

In [25]:
a

{'clickbate-prob': 0.9099950522492393,
 'notClickbate-prob': 0.09000494775076032,
 'flag': 'Looks Safe',
 'color': 'safe-color'}